# En este colab se prueba el modelo finetuneado de BNE con text encodings sobre un conjunto de 4 conversaciones de prueba

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MAF-ProyectoNLP/Test-Conversaciones/blob/main/BNE-TE%20evaluación%20conversación%20-%20desde%20archivos%20de%20audio.ipynb)


*0*. Se instalan las bibliotecas necesarias

In [2]:
!pip install datasets
!pip install transformers
!pip install SpeechRecognition pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 22.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 99.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wh

*1*. Se importan las dependencias

In [3]:
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import datasets
import numpy as np
import random
import speech_recognition as sr

import glob

from transformers import  RobertaForSequenceClassification
from transformers import RobertaTokenizer

from sklearn.metrics import accuracy_score

from pydub import AudioSegment
from pydub.silence import split_on_silence
from google.colab import files


In [4]:
torch.cuda.empty_cache()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

*2*. Definimos el modelo a utilizar, en este caso el modelo finetuneado de BNE con text enconding

In [5]:
MODEL_TYPE = "PlanTL-GOB-ES/roberta-large-bne-te"

In [6]:
MAX_LEN=256

*3*. TestDataset and CompDataset son las clases usadas para la tokenización de la frases, CompDataset se utiliza para entrenar (devuelve también el label), y TestDataset es utilizada para tokenizar las frases para validación

In [7]:
class TestDataset(Dataset):

    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):
        sentence1 = self.df_data.loc[index, 'premise']
        sentence2 = self.df_data.loc[index, 'hypothesis']

        encoded_dict = tokenizer.encode_plus(
                    sentence1, sentence2,      
                    add_special_tokens = True,  
                    max_length = MAX_LEN,           
                    pad_to_max_length = True,
                    return_attention_mask = True,   
                    return_tensors = 'pt',          
                    padding="max_length", 
                    truncation=True
               )
        
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]

        sample = (padded_token_list, att_mask)
        return sample

    def __len__(self):
        return len(self.df_data)

In [8]:
class CompDataset(Dataset):

    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):
        sentence1 = self.df_data.loc[index, 'premise']
        sentence2 = self.df_data.loc[index, 'hypothesis']

        encoded_dict = tokenizer.encode_plus(
                    sentence1, sentence2,         
                    add_special_tokens = True, 
                    max_length = MAX_LEN,  
                    pad_to_max_length = True,
                    return_attention_mask = True,
                    return_tensors = 'pt',   
                    padding="max_length", 
                    truncation=True
               )

        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        
        target = torch.tensor(self.df_data.loc[index, 'label'])

        sample = (padded_token_list, att_mask, target)
        return sample

    def __len__(self):
        return len(self.df_data) 

*4*. Se carga el modelo

In [9]:
tokenizer = RobertaTokenizer.from_pretrained(MODEL_TYPE)
model =  RobertaForSequenceClassification.from_pretrained(MODEL_TYPE, num_labels=3)

model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50262, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
         

*5* La función getlabel predice el entailment de una frase, que la recibe como una TestDataset 

In [10]:
def get_label(test_dataloader):
  total = 0

  for _, batch in enumerate(test_dataloader):

    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)  

    import os
    os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
    outputs = model(b_input_ids, attention_mask=b_input_mask)
 
    preds = outputs[0].detach().cpu().numpy()

    y_pred = np.argmax(preds, axis=1)
  
  return y_pred

 > predict_matrix calcula una matriz de entailments, indicando en la posición (i,j) el entailment de la hipotesis i sobre la premisa j



In [11]:
def predict_matrix(hypotesis_to_be_tested,answers_from_operator):
  M = np.zeros([len(hypotesis_to_be_tested),len(answers_from_operator)])
  
  for i in range(len(hypotesis_to_be_tested)):
      for j in range(len(answers_from_operator)):
        dataset = {'premise': [answers_from_operator[j]], 'hypothesis': [hypotesis_to_be_tested[i]]}

        dataloader = torch.utils.data.DataLoader(TestDataset(pd.DataFrame(dataset)), batch_size=1, shuffle=False, num_workers=1)
        M[i,j] = get_label(dataloader)

  return(M)

*6*. Se crea la función conversation_analysis, que recibe como parámetros una lista de hipótesis a ser testeadas, y las respuestas brindadas por la operadora. Se utiliza como base la función predict_matrix para indicar para cada hipótesis si se encuentra presente en las respuestas, y en el órden correcto. 



In [12]:
def conversation_analysis(hypotesis_to_be_tested,answers_from_operator):
  out=predict_matrix(hypotesis_to_be_tested,answers_from_operator)
  hypotesis_ordered= np.ones([len(hypotesis_to_be_tested)])*-1
  for h in range(len(hypotesis_to_be_tested)):
    checker= np.zeros([len(hypotesis_to_be_tested)])
    checker[h]=1
    c=out.T.dot(checker)
    for x in range(len(c)):
      if c[x]==0:
        hypotesis_ordered[h]=x
        break

  ok=[]
  wrong=[]

  for i in range(len(hypotesis_ordered)):
    if hypotesis_ordered[i] ==i:
      ok.append(hypotesis_to_be_tested[i] +" correctamente presente en la comunicación")
    elif hypotesis_ordered[i]<0:
      wrong.append(hypotesis_to_be_tested[i] +" NO PRESENTE en la comunicación")
    else:
      wrong.append(hypotesis_to_be_tested[i] +" presente en la comunicación, pero NO EN EL ORDEN CORRECTO")

  print("** Devolución de la comunicación realizado por la operadora **")
  if len(wrong)>0:
    print("A mejorar:")
  else:
    ok.append("Felicitaciones! Comunicación realizada de forma deseada")
  for w in wrong:
    print(w)

  if len(ok)>0:
    print("Positivo:")
  for o in ok:
    print(o)

*7*. **PRUEBA**

*7.1* La lista de hipótesis a testear

In [13]:
hypotesis_to_be_tested=["Saludo","Ofrecio asistencia"]

*7.2* Se descargan las conversaciones de prueba del repositorio en GitHub y para cada una se realiza en análisis

In [14]:
dirTempChunked="Audio_Chunked/"
!mkdir "Audio_Chunked"

In [15]:
!git clone "https://github.com/MAF-ProyectoNLP/Test-Conversaciones"

Cloning into 'Test-Conversaciones'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 10 (delta 5), reused 5 (delta 3), pack-reused 0
Unpacking objects: 100% (10/10), 560.61 KiB | 7.01 MiB/s, done.


> *chunk_conversation divide la conversacion entre las respuestas de la operadora y las del cliente*



In [16]:
def chunk_conversation(name,sound):
  chunks = split_on_silence(sound, min_silence_len = 500, silence_thresh = -50, seek_step=100 )

  for i, chunk in enumerate(chunks):
      chunk.export('/content/'+dirTempChunked+name+"_phrase_"+str(i)+".wav", format="wav")

> *recognize_Text recibe la ruta de un archivo .wav y aplica la biblioteca de speech recognizer para reconocer el texto en español*





In [17]:
def recognize_Text(filename):
  r = sr.Recognizer()
  with sr.AudioFile(filename) as source:
      audio_data = r.record(source)
      text = r.recognize_google(audio_data,language="es-US",show_all=False)
      return text

Para cada archivo en la carpeta se realiza el análisis y se imprime su salida

In [18]:
conversations = glob.glob("Test-Conversaciones" + '/*.wav')

for conv in conversations:
  name=conv.split("/")[1]
  print(name)
  chunk_conversation(name,AudioSegment.from_file(conv))
  answers_from_operator=[]

  answers_from_operator.append(recognize_Text(filename="/content/"+dirTempChunked+name+"_phrase_0.wav"))
  answers_from_operator.append(recognize_Text(filename="/content/"+dirTempChunked+name+"_phrase_2.wav"))
  print("Respuestas detectadas en el audio:")
  print(answers_from_operator)
  print("\n")
  conversation_analysis(hypotesis_to_be_tested,answers_from_operator)
  print("\n\n---------------------------------------------------------")



conversacion1.wav
result2:
{   'alternative': [   {'confidence': 0.94842249, 'transcript': 'hola'},
                       {'transcript': 'o'},
                       {'transcript': 'ola'}],
    'final': True}
result2:
{   'alternative': [   {'confidence': 0.958049, 'transcript': 'en que puedo'},
                       {'transcript': 'en que puedo ayudar'},
                       {'transcript': 'en que puedo ayuda'}],
    'final': True}
Respuestas detectadas en el audio:
['hola', 'en que puedo']


** Devolución de la comunicación realizado por la operadora **
A mejorar:
Ofrecio asistencia NO PRESENTE en la comunicación
Positivo:
Saludo correctamente presente en la comunicación


---------------------------------------------------------
conversacion3.wav
result2:
{   'alternative': [   {'confidence': 0.958049, 'transcript': 'en que puedo'},
                       {'transcript': 'en que puedo ayudar'},
                       {'transcript': 'en que puedo ayuda'}],
    'final': True}
resul

TODO --- Ver si dejar mostrando que sin ¿? no funciona bien, o acomodar los audios para que funcione

In [19]:
answers_from_operator=["Hola", "¿En que puedo ayudar?"]
conversation_analysis(hypotesis_to_be_tested,answers_from_operator)

** Devolución de la comunicación realizado por la operadora **
Positivo:
Saludo correctamente presente en la comunicación
Ofrecio asistencia correctamente presente en la comunicación
Felicitaciones! Comunicación realizada de forma deseada


In [20]:
answers_from_operator=["Hola", "que cansancio"]
conversation_analysis(hypotesis_to_be_tested,answers_from_operator)

** Devolución de la comunicación realizado por la operadora **
A mejorar:
Ofrecio asistencia NO PRESENTE en la comunicación
Positivo:
Saludo correctamente presente en la comunicación


In [21]:
answers_from_operator=["¿En que puedo ayudar?","Hola"]
conversation_analysis(hypotesis_to_be_tested,answers_from_operator)

** Devolución de la comunicación realizado por la operadora **
A mejorar:
Saludo presente en la comunicación, pero NO EN EL ORDEN CORRECTO
Ofrecio asistencia presente en la comunicación, pero NO EN EL ORDEN CORRECTO


In [22]:
answers_from_operator=["muy buenas noches","Cómo se encuentra el día de hoy?"] 
conversation_analysis(hypotesis_to_be_tested,answers_from_operator)

** Devolución de la comunicación realizado por la operadora **
A mejorar:
Ofrecio asistencia NO PRESENTE en la comunicación
Positivo:
Saludo correctamente presente en la comunicación
